In [1]:
from glob import glob
import os

import cv2
import numpy as np
import torch
from PIL import Image
from pdf2image import convert_from_path
from imutils import resize
from tqdm import tqdm
from scoring import *
from border_and_title import *

good = list(glob("../data/original/Good/*.pdf"))
ugly = list(glob("../data/original/Ugly/*.pdf"))

len(good), len(ugly)

(16, 25)

In [2]:
def process(files, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2

    progress = tqdm(total=len(files))

    for f in files:
        pages = convert_from_path(f, dpi=300)
        img = np.array(pages[0])
        img = im_resize(img)
        img_comp = img.copy()
        im_h, im_w, _ = img.shape

        sorted_indices, contours, hierarchy_area = detect_objects(img)
        border_1, border_2 = detect_borders(contours, sorted_indices)
        x, y, w, h = border_1
        cv2.rectangle(img_comp, (x, y), (x+w, y+h), color, 10)
        x, y, w, h = border_2
        cv2.rectangle(img_comp, (x, y), (x+w, y+h), color, 10)

        title_contours, line_90 = detect_probable_title_sections(img)

        bb = []
        for cnt in title_contours:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = w / float(h)
            area = w * h

            if area > 7000 and 1.3 < aspect_ratio < 10:
                if y > int((im_h * 70) / 100) and y+h > int((im_h * 93) / 100):
                    # cv2.rectangle(img_comp, (x, y), (x + w, y + h), (255, 0, 0), 10)
                    bb.append((x, y, x+w, y+h))
                elif y > int((im_h * 80) / 100) and y+h > int((im_h * 90) / 100):
                    # cv2.rectangle(img_comp, (x, y), (x + w, y + h), (255, 0, 0), 10)
                    bb.append((x, y, x+w, y+h))

        try:
            bb = np.array(bb)

            if line_90 is not None:
                x1, y1, x2, y2 = line_90
                # cv2.line(img_comp, (x1, y1), (x2, y2), color, 5)
                cv2.rectangle(img_comp, (x1, y1), (x2, im_h), color, 10)
            else:
                cv2.rectangle(img_comp, (bb[:, 0].min(), bb[:, 1].min()), (bb[:, 2].max(), im_h), color, 10)
        except IndexError:
            print(f)
            print(bb)

        cv2.imwrite(f"{output_dir}/{os.path.basename(f)}.jpg", img_comp)
        progress.update(1)

In [3]:
process(ugly, "../data/border-and-title/ugly")
process(good, "../data/border-and-title/good")

 28%|██▊       | 7/25 [00:06<00:17,  1.02it/s]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 96%|█████████▌| 24/25 [00:21<00:00,  1.21it/s]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 38%|███▊      | 6/16 [00:04<00:07,  1.40it/s]

../data/original/Good/Apple.pdf
[]


100%|██████████| 16/16 [00:15<00:00,  1.05it/s]
